In [1]:
import kaggle

In [2]:
!mv kaggle.json /root/.kaggle

mv: cannot stat 'kaggle.json': No such file or directory


In [3]:
import kaggle

In [4]:
!kaggle competitions download -c dogs-vs-cats

dogs-vs-cats.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
!unzip -q dogs-vs-cats.zip

replace test1.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
A
A


In [6]:
!unzip -q train.zip

replace train/cat.0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [10]:
# Image Data Generator
from tensorflow import keras
idg = keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0, validation_split=0.1)

In [11]:
import pandas as pd
df = pd.DataFrame(columns= ["imagePath", "label"])

In [13]:
import os
df["imagePath"] = os.listdir("train")

In [14]:
df["label"] = df.imagePath.str.split(".", expand=True)[0]

In [15]:
df.head()

,imagePath,label
0,cat.2948.jpg,cat
1,cat.3971.jpg,cat
2,dog.9765.jpg,dog
3,cat.6850.jpg,cat
4,cat.768.jpg,cat


In [16]:
batch_size=64

In [17]:
train_idg = idg.flow_from_dataframe(dataframe=df, directory="train/", x_col="imagePath", y_col="label", 
                                    target_size=(150, 150), batch_size=batch_size,
                                    subset="training")

Found 22500 validated image filenames belonging to 2 classes.


In [18]:
val_idg = idg.flow_from_dataframe(dataframe=df, directory="train/", x_col="imagePath", y_col="label", 
                                    target_size=(150, 150), batch_size=batch_size,
                                    subset="validation")

Found 2500 validated image filenames belonging to 2 classes.


In [19]:
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import *


In [20]:
base_model = VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))

In [21]:
for layer in base_model.layers:
  Layer.trainable = False

In [22]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 21,137,729
Trainable params: 0
Non-trainable params: 21,137,729
_________________________________________________________________


In [26]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])


In [27]:
model.fit(train_idg,epochs=10,batch_size=64,validation_data=(val_idg))

ValueError: ignored